In [ ]:
"""Implementing the Actual Decision Tree and Prining the Information About each
   Node """

In [28]:
#Importing the Necessary Libraries for Creating a Decision Tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import math
#This function is Used to convert data from being continuous to Discrete with classes "1","2","3","4"
def fun2(data,a1,a2,a3):
    if(data<a1):
        return "1"
    if(data<a2):
        return "2"
    if(data<a3):
        return "3"
    else:
        return "4"
#This function is used to Convert data from continuous to descrete. This is done by placing the value on the bases of mean, the mean of min_value
# and mean, mean of max_val and mean. This replaces the Value in Place.
def fun1(data,column):
    min_val=data[column].min()
    n1=data[column].mean()
    n2=(n1+min_val)/2
    max_val=data[column].max()
    n3=(max_val+n1)/2
    arr=[]
    arr.append(n2)
    arr.append(n1)
    arr.append(n3)
    data[column]=data[column].apply(fun2,args=arr)
    
#Loading the iris data from datasets in sklearn    
iris=load_iris()
x_iris=iris.data
#taking the x value or features from the data
y_iris=iris.target
#taking the y value or output from the data
x_data=pd.DataFrame(x_iris)
#Renaming the columns for convinience
x_data.columns=["sepal_length","sepal_width","petal_length","petal_width"]
#Making the data in each column descrete
fun1(x_data,"sepal_length")
fun1(x_data,"sepal_width")
fun1(x_data,"petal_length")
fun1(x_data,"petal_width")

In [29]:
class Node:
    def __init__(self,split_data,out,level):
        self.split_data=split_data
        #This is used to store the data of the split_feature that has be used to split the Decision Tree here. It is defined as Null
        #or None for Leaf Node
        self.children={}
        #The children represents the Node that are connected to the particular Node and This has key with the feature_value that
        #it has been split upon and the value as the child Node
        self.level=level
        #This is used to store the level of the data which is present upon 
        self.out=out
        #This helps to store the output that will be predicted for this particular node
        self.entropy=0.0
        #Entropy for the Particular Node
        self.gainRatio=-1
        #Gain Ratio for that Node
        
    def add(self,feature_val,object):
        self.children[feature_val]=object
        #This is Used to add the child node based on the feature value to a node 

In [30]:
#Defining the Optimum Functions for Calculating the split feature

#This function is used to Calculate the Frequency of the particular class in the output dataset
def freq(y):
    d1={}    #Dictionary to store the Frequency of each class
    for x in y: 
        if(x in d1):
            d1[x]=d1[x]+1
        else:
            d1[x]=1
    return d1
#Return the dictionary with key as the classes present in the data and the count of each class as the value

#This Function is used to calculate the entropy of the data 
def entropy(y):
    d1=freq(y) #Calculating Frequency of each class in the data
    size=len(y) # 
    ent=0
    for x in d1:
        test=d1[x]/size
        ent+=(-1*(test))*math.log2(test)
    return ent

#This is used to calculate the gain Ratio which will be further used in splitting the data as gain ratio is the deciding factor for spliitng the data
def gainRatio(x,y,feature):
    ent1=entropy(y) #Defining the Orignal entropy before split
    ent2=0 #Initialising the entropy for split data
    values=set(x[:,feature])
    split=0 #initialising the split ratio
    frame=pd.DataFrame(x) 
    frame[frame.shape[1]]=y #Adding the Result Value to the data
    size=frame.shape[0]
    #Running the loop for calculating info for each of the classes in the split feature i.e feature
    for x in values:
        datax=frame[frame[feature]==x] #Getting the data with particular class
        size1=datax.shape[0]
        ent2+=(size1/size)*entropy(datax[datax.shape[1]-1]) #Calculating the Information for that particular Class
        split+=(-size1/size)*math.log2(size1/size) #Calculating the split_info for that class
    gain=ent1-ent2 #Calculating the Information Gain
    ratio=gain/split #Calculating the Split Ratio
    return ratio #Returns the Gain Ratio

In [31]:
#Implementing functions for Prediction of data
def predict(x,root):
    y=[]
    #Loop for prediction of data
    for i in range(len(x)):
        y.append(0) #Appending default value zero to data
        y[i]=pred(x[i],root) #Predicting data for each point
    return y #Returning the predicting y
def pred(xpoint,node):
    if(len(node.children)==0): #If len of children is zero or reached leaf node
        return node.out #Returning the predicted output for that node
    
    feature=xpoint[node.split_data] #Getting the split feature of the Node
    if(feature in node.children): #If class present in the child Node then recursively
        #calling the function with childNode
        return pred(xpoint,node.children[feature])
    else: #Else returning the output for that Node
        return node.out 

In [37]:
#Implementation for Formation of Decision Tree

def decisionTree(x,y,features,classes,level):
    if(len(features)==0): #First Base Condition When No Features are left that can be Used to Split the Decision Tree
        out="" #Output the Node will Predict
        max_count=-1 #Initialising the Count for the freq of class in data as Majority class will be the output
        freqs=freq(y) #Calculating the Frequency of different classed in target
        print("Level",level) #Printing the Current Level Of the Tree 
        #This loop is used to print the count of classes in target and used to select the one with the max count which will be 
        #Output
        for i in classes:
            if(not(i in freqs)):
                print("Count of ",i,"=",0) 
            else:
                if(freqs[i]>max_count):
                    max_count=freqs[i]
                    out=i #The Out Value for this node
                print("Count of",i,"=",freqs[i])
        print("Current Entropy is =",entropy(y)) #Printing the Entropy for current Node
        print("Reached Leaf Node") #Print the ent of tree or leaf Node
        print()
        t1=Node(None,out,level) #return Node with level and ouput
        t1.entropy=entropy(y) #storing the entropy in the Node
        return t1 #Returning the Node
    
    if(len(set(y))==1):
        print("Level",level) #Printing the Level of the Node
        out="" #The ouput for the Node
        #This loop is used to print the count of classes in target and used to select the class that is the only class in target 
        for i in classes:
            if(i in y):
                out=i #Out Value for this Node
                print("Count of",i,"=",len(y))
            else:
                print("Count of",i,"=",0)
                
        print("Current Entropy is = 0.0") #This is the Entropy for the Current Node
        print("Reached Leaf Node") #Printing that the Tree has reached the end or leaf Node
        print()
        t1=Node(None,out,level) # Creating the leaf Node 
        return t1 #Returning the Node
    
    gain=-9999999999 #Initialising the Gain with some value
    feature=None #Feature that will be used to split
    #This loop traverses through the features and calculates the Gain for each of them and compares. The Feature which results
    #Maximum Gain ratio is selected and feature represents the splitting feature
    for i in features:
        cgain=gainRatio(x,y,i)
        if(cgain>gain):
            gain=cgain
            feature=i
    print("Level",level) #Printing the Level of the Node in the Tree
    freqs=freq(y) #Calculating the Frequency For each class in target data
    count=-1 
    out=""
    #This loop is used to print the count of classes in target and used to select the one with the max count which will be 
    #Output
    for i in classes:
        if(i in freqs):
            print("Count of ",i,"=",freqs[i])
            if(freqs[i]>count):
                count=freqs[i]
                out=i
        else:
            print("Count of ",i,"= 0")
    print("Current Entropy is =",entropy(y)) #Entropy for the Current Node
    print("Splitting on feature",x_data.columns[feature],"With Gain Ratio",gain) #This Prints the feature on which the data 
    #is split
    print("")
    
    split_class=set(x[:,feature])#Getting the Various Classes in the feature which will be used to split the data
    frame=pd.DataFrame(x) #Creating a DataFrame for the numpy data
    frame[frame.shape[1]]=y #Joing the output to the data
    c_Node=Node(feature,out,level) #Creating a Node with the data, output precited for that Node and the level
    c_Node.entropy=entropy(y) #Assigning the Entropy value to the Node
    c_Node.gainRatio=gainRatio(x,y,feature) # Assigning the Gain Ratio to the Node
    features.remove(feature) #Removing the Feature that was used to split the data as the a feature can be only used once
    #Running a for loop for classes in the feature which was used to split the data so as to sort the data based on that class
    # and Recursively call the Decision Tree Function.
    for i in split_class:
        n_frame=frame[frame[feature]==i] #Data with the Particular class
        x_new=n_frame.iloc[:,0:n_frame.shape[1]-1].values #The x data for the recursive call with only selected class from feature
        y_new=n_frame.iloc[:,n_frame.shape[1]-1].values #The y data for the same
        ch_Node=decisionTree(x_new,y_new,features,classes,level+1) 
        #Recursively calling the function with incremented Level
        #and new x and y and feature arra
        c_Node.add(i,ch_Node) #Adding this Node to the child of data
    
    return c_Node #Returning the Node 
        
#Function to start the Formation of Decision Tree
def fit(x,y):
    split_features=[] #Array that stores the indexes of feature in data 
    for i in range(len(x[0])):
        split_features.append(i)
    classes=set(y) #Classes Present in the Output data Here 0,1,2
    root=decisionTree(x,y,split_features,classes,0) #Function to form Decision Tree 
    return root #Returns the Root of the Tree.

x_train,x_test,y_train,y_test=train_test_split(x_data.values,y_iris,random_state=100)
root=fit(x_train,y_train)
y=predict(x_test,root)
count=0
for i in range(len(y_test)):
    if(y_test[i]==y[i]):
        count=count+1
print(count,len(y_test))
print("The Score for the Data is",count/len(y_test))

Level 0
Count of  0 = 36
Count of  1 = 40
Count of  2 = 36
Current Entropy is = 1.583143101527774
Splitting on feature petal_width With Gain Ratio 0.7068646591932941

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 25
Current Entropy is = 0.0
Reached Leaf Node

Level 1
Count of 0 = 0
Count of 1 = 7
Count of 2 = 0
Current Entropy is = 0.0
Reached Leaf Node

Level 1
Count of 0 = 36
Count of 1 = 0
Count of 2 = 0
Current Entropy is = 0.0
Reached Leaf Node

Level 1
Count of  0 = 0
Count of  1 = 33
Count of  2 = 11
Current Entropy is = 0.8112781244591328
Splitting on feature petal_length With Gain Ratio 0.45551250909754704

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 6
Current Entropy is = 0.0
Reached Leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Entropy is = 0.0
Reached Leaf Node

Level 2
Count of  0 = 0
Count of  1 = 32
Count of  2 = 5
Current Entropy is = 0.5713549744279549
Splitting on feature sepal_length With Gain Ratio 0.12275198489935117

Level 3
C